In [9]:
import urllib.request
import re
import os
import urllib


#根据给定的网址来获取网页详细信息
def getHtml(url):
    page = urllib.request.urlopen(url)
    html = page.read()
    return html.decode('UTF-8')


def getImg(html):
    #在整个网页中过滤出所有图片的地址，放在imglist中
    reg = r'src="(.+?\.jpg)" pic_ext'
    imgre = re.compile(reg)
    imglist = imgre.findall(html)
    
    x = 0
    #将图片保存到文件夹中
    path = './pictures'
    if not os.path.isdir(path): 
        os.makedirs(path) 
    paths = path+'\\'
    
    #打开imglist中保存的图片网址，并下载图片保存在本地，format格式化字符串
    for imgurl in imglist: 
        urllib.request.urlretrieve(imgurl,'{0}{1}.jpg'.format(paths,x)) 
        x = x + 1
    return imglist

html = getHtml("http://tieba.baidu.com/p/2460150866")
getImg(html) #从网页源代码中分析并下载保存图片

['https://imgsa.baidu.com/forum/w%3D580/sign=294db374d462853592e0d229a0ee76f2/e732c895d143ad4b630e8f4683025aafa40f0611.jpg',
 'https://imgsa.baidu.com/forum/w%3D580/sign=941c6a9596dda144da096cba82b6d009/e889d43f8794a4c2e5d529ad0ff41bd5ac6e3947.jpg',
 'https://imgsa.baidu.com/forum/w%3D580/sign=750661a0fcfaaf5184e381b7bc5594ed/75fafbedab64034fc3ed0b80aec379310a551d11.jpg',
 'https://imgsa.baidu.com/forum/w%3D580/sign=114c0f68b58f8c54e3d3c5270a282dee/3d4e78f0f736afc3c72cf6e3b219ebc4b7451211.jpg',
 'https://imgsa.baidu.com/forum/w%3D580/sign=3b5751825882b2b7a79f39cc01accb0a/9ac37d1ed21b0ef481c651d8dcc451da81cb3e0f.jpg',
 'https://imgsa.baidu.com/forum/w%3D580/sign=26c16593bd315c6043956be7bdb0cbe6/894443a98226cffcd8287634b8014a90f603ea3c.jpg',
 'https://imgsa.baidu.com/forum/w%3D580/sign=8905fff979f0f736d8fe4c093a54b382/08d2d539b6003af318e615e6342ac65c1138b6dd.jpg',
 'https://imgsa.baidu.com/forum/w%3D580/sign=c6335d3d86d6277fe912323018391f63/9dcd7cd98d1001e97833dd89b90e7bec54e7972e.jpg',


In [11]:
# -*- coding: utf-8 -*-

## 载入所需库
import cv2
import time
 
def style_transfer(pathIn='',
                   pathOut='',
                   model='',
                   width=None,
                   jpg_quality=80):
    
    #pathIn: 原始图片的路径
    #pathOut: 风格化图片的保存路径
    #model: 预训练模型的路径
    #width: 设置风格化图片的宽度，默认为None, 即原始图片尺寸
    #jpg_quality: 0-100，设置输出图片的质量，默认80，越大图片质量越好

    
    ## 读入原始图片，调整图片至所需尺寸，然后获取图片的宽度和高度
    img = cv2.imread(pathIn)
    (h, w) = img.shape[:2]
    if width is not None:
        img = cv2.resize(img, (width, round(width*h/w)), interpolation=cv2.INTER_CUBIC)
        (h, w) = img.shape[:2]
    
    ## 从本地加载预训练模型
    net = cv2.dnn.readNetFromTorch(model)
    print('已加载预训练模型')
    
    ## 将图片构建成一个blob：设置图片尺寸，将各通道像素值减去平均值（比如ImageNet所有训练样本各通道统计平均值）
    blob = cv2.dnn.blobFromImage(img, 1.0, (w, h), 
                                 (103.939, 116.779, 123.680), 
                                 swapRB=False, 
                                 crop=False)
    
    ## 执行一次前馈网络计算，并输出计算所需的时间
    net.setInput(blob)
    start = time.time()
    output = net.forward()
    end = time.time()
    print("风格迁移花费：{:.2f}秒".format(end - start))

    ## reshape输出结果, 将减去的平均值加回来，并交换各颜色通道
    output = output.reshape((3, output.shape[2], output.shape[3]))
    output[0] += 103.939
    output[1] += 116.779
    output[2] += 123.680
    output = output.transpose(1, 2, 0)
    
    ## 输出风格化后的图片
    cv2.imwrite(pathOut, output, [int(cv2.IMWRITE_JPEG_QUALITY), jpg_quality])
    

   
## 测试
import glob
models = glob.glob('./*/*/*.t7')
models      ## 列出所有可用的预训练模型 

['.\\models\\eccv16\\composition_vii.t7',
 '.\\models\\eccv16\\la_muse.t7',
 '.\\models\\eccv16\\starry_night.t7',
 '.\\models\\eccv16\\the_wave.t7',
 '.\\models\\instance_norm\\candy.t7',
 '.\\models\\instance_norm\\feathers.t7',
 '.\\models\\instance_norm\\la_muse.t7',
 '.\\models\\instance_norm\\mosaic.t7',
 '.\\models\\instance_norm\\starry_night.t7',
 '.\\models\\instance_norm\\the_scream.t7',
 '.\\models\\instance_norm\\udnie.t7']

In [12]:
for x in range(0,46):
    pathIn = ('./pictures/{0}.jpg'.format(x))
    pathOut = ('./result_starry/{0}.jpg'.format(x))
    model = './models/instance_norm/starry_night.t7'
    style_transfer(pathIn, pathOut, model, width=500)

已加载预训练模型
风格迁移花费：1.07秒
已加载预训练模型
风格迁移花费：1.01秒
已加载预训练模型
风格迁移花费：0.95秒
已加载预训练模型
风格迁移花费：1.17秒
已加载预训练模型
风格迁移花费：1.07秒
已加载预训练模型
风格迁移花费：1.07秒
已加载预训练模型
风格迁移花费：1.20秒
已加载预训练模型
风格迁移花费：1.13秒
已加载预训练模型
风格迁移花费：1.28秒
已加载预训练模型
风格迁移花费：1.31秒
已加载预训练模型
风格迁移花费：1.16秒
已加载预训练模型
风格迁移花费：1.25秒
已加载预训练模型
风格迁移花费：1.13秒
已加载预训练模型
风格迁移花费：1.29秒
已加载预训练模型
风格迁移花费：1.29秒
已加载预训练模型
风格迁移花费：1.14秒
已加载预训练模型
风格迁移花费：1.21秒
已加载预训练模型
风格迁移花费：1.25秒
已加载预训练模型
风格迁移花费：1.13秒
已加载预训练模型
风格迁移花费：1.11秒
已加载预训练模型
风格迁移花费：1.15秒
已加载预训练模型
风格迁移花费：1.33秒
已加载预训练模型
风格迁移花费：1.25秒
已加载预训练模型
风格迁移花费：1.29秒
已加载预训练模型
风格迁移花费：1.11秒
已加载预训练模型
风格迁移花费：1.15秒
已加载预训练模型
风格迁移花费：1.15秒
已加载预训练模型
风格迁移花费：1.13秒
已加载预训练模型
风格迁移花费：1.16秒
已加载预训练模型
风格迁移花费：1.18秒
已加载预训练模型
风格迁移花费：1.24秒
已加载预训练模型
风格迁移花费：1.15秒
已加载预训练模型
风格迁移花费：1.37秒
已加载预训练模型
风格迁移花费：1.48秒
已加载预训练模型
风格迁移花费：1.33秒
已加载预训练模型
风格迁移花费：1.28秒
已加载预训练模型
风格迁移花费：1.22秒
已加载预训练模型
风格迁移花费：1.52秒
已加载预训练模型
风格迁移花费：1.56秒
已加载预训练模型
风格迁移花费：1.49秒
已加载预训练模型
风格迁移花费：1.12秒
已加载预训练模型
风格迁移花费：1.68秒
已加载预训练模型
风格迁移花费：1.31秒
已加载预训练模型
风格迁移花费：1.46秒
已加载预训练模型
风格迁移花费：1.07秒
已加载预训练模型
风

In [14]:
for x in range(0,46):
    pathIn = ('./pictures/{0}.jpg'.format(x))
    pathOut = ('./result_scream/{0}.jpg'.format(x))
    model = './models/instance_norm/the_scream.t7'
    style_transfer(pathIn, pathOut, model, width=500)

已加载预训练模型
风格迁移花费：0.40秒
已加载预训练模型
风格迁移花费：0.44秒
已加载预训练模型
风格迁移花费：0.42秒
已加载预训练模型
风格迁移花费：0.50秒
已加载预训练模型
风格迁移花费：0.54秒
已加载预训练模型
风格迁移花费：0.50秒
已加载预训练模型
风格迁移花费：0.49秒
已加载预训练模型
风格迁移花费：0.39秒
已加载预训练模型
风格迁移花费：0.39秒
已加载预训练模型
风格迁移花费：0.41秒
已加载预训练模型
风格迁移花费：0.41秒
已加载预训练模型
风格迁移花费：0.42秒
已加载预训练模型
风格迁移花费：0.38秒
已加载预训练模型
风格迁移花费：0.48秒
已加载预训练模型
风格迁移花费：0.49秒
已加载预训练模型
风格迁移花费：0.48秒
已加载预训练模型
风格迁移花费：0.50秒
已加载预训练模型
风格迁移花费：0.52秒
已加载预训练模型
风格迁移花费：0.44秒
已加载预训练模型
风格迁移花费：0.45秒
已加载预训练模型
风格迁移花费：0.45秒
已加载预训练模型
风格迁移花费：0.50秒
已加载预训练模型
风格迁移花费：0.50秒
已加载预训练模型
风格迁移花费：0.65秒
已加载预训练模型
风格迁移花费：0.50秒
已加载预训练模型
风格迁移花费：0.46秒
已加载预训练模型
风格迁移花费：0.48秒
已加载预训练模型
风格迁移花费：0.52秒
已加载预训练模型
风格迁移花费：0.48秒
已加载预训练模型
风格迁移花费：0.56秒
已加载预训练模型
风格迁移花费：0.73秒
已加载预训练模型
风格迁移花费：0.63秒
已加载预训练模型
风格迁移花费：0.74秒
已加载预训练模型
风格迁移花费：0.74秒
已加载预训练模型
风格迁移花费：0.65秒
已加载预训练模型
风格迁移花费：0.56秒
已加载预训练模型
风格迁移花费：0.47秒
已加载预训练模型
风格迁移花费：0.72秒
已加载预训练模型
风格迁移花费：0.65秒
已加载预训练模型
风格迁移花费：0.76秒
已加载预训练模型
风格迁移花费：0.45秒
已加载预训练模型
风格迁移花费：0.75秒
已加载预训练模型
风格迁移花费：0.57秒
已加载预训练模型
风格迁移花费：0.67秒
已加载预训练模型
风格迁移花费：0.44秒
已加载预训练模型
风